# Day 0: Imports and Utility Functions

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import re
import numpy as np
import random
import string
from collections import Counter, defaultdict, namedtuple, deque, OrderedDict
from functools   import lru_cache, reduce
from statistics  import mean, median, mode, stdev, variance
from itertools   import (permutations, combinations, groupby, cycle, chain, zip_longest, takewhile, dropwhile, count as count_from)
from heapq       import heappush, heappop
from operator    import iand, ior, ilshift, irshift

# Day 1: Not Quite Lisp

In [2]:
with open('inputs/day1.txt') as f:
    content = f.read()
print(content.count('(') - content.count(')'))
cnt = 0
for idx, ch in enumerate(content, 1):
    if ch == '(':
        cnt += 1
    else:
        cnt -= 1
        if cnt < 0:
            print(idx)
            break

232
1783


# Day 2: I Was Told There Would Be No Math

In [3]:
paper = 0
ribbon = 0
with open('inputs/day2.txt') as f:
    for line in f:
        a, b, c = tuple(map(int, line.split('x')))
        ribbon += (sum((a, b, c)) - max(a, b, c)) * 2 + a * b * c
        s1, s2, s3 = a * b, a * c, b * c
        paper += min(s1, s2, s3) + sum((s1, s2, s3)) * 2
print(paper)
print(ribbon)

1606483
3842356


# Day 3: Perfectly Spherical Houses in a Vacuum

In [4]:
with open('inputs/day3.txt') as f:
    content = ''.join([line.strip() for line in f])
# print(content)
direction = {'>' : (0, 1), '<' : (0, -1), '^' : (1, 0), 'v' : (-1, 0)}
i, j = 0, 0
visited = set([(i, j)])
for ch in content:
    di, dj = direction[ch]
    i, j = i + di, j + dj
    visited.add((i, j))
print(len(visited))
i, j = 0, 0
visited = set([(i, j)])
for ch in content[::2]:
    di, dj = direction[ch]
    i, j = i + di, j + dj
    visited.add((i, j))
i, j = 0, 0
for ch in content[1::2]:
    di, dj = direction[ch]
    i, j = i + di, j + dj
    visited.add((i, j))
print(len(visited))

2592
2360


# Day 4: The Ideal Stocking Stuffer

In [5]:
import hashlib

def valid_md5(s):
    m = hashlib.md5(s.encode('utf-8'))
    code = m.hexdigest()
    return len(code) >= 5 and all(code[i] == '0' for i in range(5))

s = "iwrupvqb"
# s = "pqrstuv"
print(next(dropwhile(lambda x : not valid_md5(s + str(x)), count_from(0))))

346386


# Day 5: Doesn't He Have Intern-Elves For This?
Old rule
* It contains at least three vowels (aeiou only), like aei, xazegov, or aeiouaeiouaeiou.
* It contains at least one letter that appears twice in a row, like xx, abcdde (dd), or aabbccdd (aa, bb, cc, or dd).
* It does not contain the strings ab, cd, pq, or xy, even if they are part of one of the other requirements.

New rule

* It contains a pair of any two letters that appears at least twice in the string without overlapping, like xyxy (xy) or aabcdefgaa (aa), but not like aaa (aa, but it overlaps).
* It contains at least one letter which repeats with exactly one letter between them, like xyx, abcdefeghi (efe), or even aaa.

In [6]:
def is_nice(s):
    return (sum(ch in 'aeiou' for ch in s) >= 3 
            and any(len(list(g)) >= 2 for key, g in groupby(s)) 
            and all(item not in s for item in ('ab', 'cd', 'pq', 'xy')))
cnt = 0
with open('inputs/day5.txt') as f:
    for line in f:
        cnt += is_nice(line)
print(cnt)

238


In [7]:
def is_nice_new(s):
    d = {}
    flag = False
    for i in range(len(s) - 1):
        x = s[i : i + 2]
        if x in d:
            if i - d[x] > 1:
                flag = True
                break
        else:
            d[x] = i
    return flag and any(s[i] == s[i + 2] for i in range(len(s) - 2))
cnt = 0
with open('inputs/day5.txt') as f:
    for line in f:
        cnt += is_nice_new(line)
print(cnt)

69


# Day 6: Probably a Fire Hazard

In [8]:
lights = np.zeros((1000, 1000))
def parse_coords(lst):
    a, b = lst[0].split(',')
    c, d = lst[1].split(',')
    return tuple(map(int, (a, b, c, d)))

with open('inputs/day6.txt') as f:
    for line in f:
        lst = re.findall('[0-9]+,[0-9]+', line)
        a, b, c, d = parse_coords(lst)
        if line.startswith('turn off'):
            lights[a:c + 1, b:d + 1] = 0
        elif line.startswith('turn on'):
            lights[a:c + 1, b:d + 1] = 1
        else:
            lights[a:c + 1, b:d + 1] = 1 - lights[a:c + 1, b:d + 1]
print(int(np.sum(lights)))

400410


In [9]:
lights = np.zeros((1000, 1000))
def turn_off(x):
    return max(x - 1, 0)
v_turn_off = np.vectorize(turn_off)
with open('inputs/day6.txt') as f:
    for line in f:
        lst = re.findall('[0-9]+,[0-9]+', line)
        a, b, c, d = parse_coords(lst)
        if line.startswith('turn off'):
            lights[a:c + 1, b:d + 1] = v_turn_off(lights[a:c + 1, b:d + 1])
        elif line.startswith('turn on'):
            lights[a:c + 1, b:d + 1] += 1
        else:
            lights[a:c + 1, b:d + 1] += 2
print(int(np.sum(lights)))

15343601


# Day 7: Some Assembly Required

In [10]:
operators = {'OR' : ior, 'AND' : iand, 'NOT' : lambda x : ~x, 'LSHIFT' : ilshift, 'RSHIFT' : irshift}
wires = {}
def digitify(s):
    return int(s) if s.isdigit() else s
lines = [] # lines of dictionary
with open('inputs/day7-1.txt') as f:
    for line in f:
        d = {}
        lh, rh = line.split(' -> ')
        rh = rh.strip('\n')
        rh = digitify(rh)
        d['rh'] = rh
        if line.startswith('NOT'):
            op, lh1 = lh.split(' ')
            lh = [digitify(lh1)]
        elif ' ' not in lh:
            op = None
            lh = [digitify(lh)]
        else:
            lh1, op, lh2 = lh.split(' ')
            lh = [digitify(lh1), digitify(lh2)]
        d['op'] = op
        d['lh'] = lh
        lines.append(d)
def is_valid(d):
    return all(( lh in wires or type(lh) == int) for lh in d['lh'])
nlines = len(lines)
used = [False] * nlines
cnt = 0
while cnt < nlines:
    cnt += 1
    for i, d in enumerate(lines):
        if used[i]: continue
        if is_valid(d):
            break
    used[i] = True
    rh = d['rh']
    lh = d['lh']
    op = d['op']
    if op is None:
        wires[rh] = lh[0] if type(lh[0]) == int else wires[lh[0]]
    elif op == 'NOT':
        wires[rh] = ~lh[0] if type(lh[0]) == int else ~wires[lh[0]]
    else:
        v1, v2 = lh[0] if type(lh[0]) == int else wires[lh[0]], lh[1] if type(lh[1]) == int else wires[lh[1]]
        wires[rh] = operators[op](v1, v2)
print(wires['a'])

14710


# Day 8: Matchsticks

In [11]:
def count_characters(line):
    cnt = 2
    i = 0
    n = len(line)
    while i < n:
        if line[i] == '\\':
            if line[i + 1] in ('"', '\\'):
                i += 2
                cnt += 1
            elif line[i + 1] == 'x':
                i += 4
                cnt += 3
            else:
                i += 1
        else:
            i += 1
    return cnt

In [12]:
with open('inputs/day8.txt', 'r') as f:
    print(sum(count_characters(line.strip()) for line in f))

1333
